In [1]:
import pandas as pd
from datetime import datetime as dtm

class UserItemData:  
  def __init__(self, path, from_date="0", to_date="0", min_ratings = -1):
    self.path = path
    self.from_date = from_date
    self.to_date = to_date
    self.min_ratings = min_ratings
    self.dfx = pd.read_table(path)
    #full date in a new column
    self.dfx["date"] = self.dfx["date_day"].astype(str) + "/" + self.dfx["date_month"].astype(str) + "/" + self.dfx["date_year"].astype(str)
    self.dfx["date"] = pd.to_datetime(self.dfx["date"], format='%d/%m/%Y') 
    self.df = self.read()
  def nratings(self):
    #print(type(self.df.shape[0]))  
    #print(self.df.shape[0])
    return int(self.df.shape[0])
  def read(self):
      #if the date is not given it sets it to the smallest possible date in dataframe
    if self.from_date == "0":
        datefr = str(self.dfx['date_day'].min())+ "/"+ str(self.dfx['date_month'].min())+"/"+str(self.dfx['date_year'].min())
        datef = pd.to_datetime(datefr, format='%d/%m/%Y')
    else:
        datefr = self.from_date.replace(".", "/")
        datef = pd.to_datetime(datefr, format='%d/%m/%Y')
        #print(datef)
    if self.to_date == "0":
        datetm = str(self.dfx['date_day'].max())+ "/"+ str(self.dfx['date_month'].max())+"/"+str(self.dfx['date_year'].max())
        datet = pd.to_datetime(datetm, format='%d/%m/%Y')
    else:
        datetm = self.to_date.replace(".", "/")
        datet = pd.to_datetime(datetm, format='%d/%m/%Y')
        #print(datet)
    if self.min_ratings == -1:    
        mask = (self.dfx['date'] >= datef) & (self.dfx['date'] <= datet)
    else:
        #self.dfx['num_ratings'] = self.dfx['movieID'].map(self.dfx['movieID'].value_counts())
        self.dfx['num_ratings'] = self.dfx.groupby('movieID')['movieID'].transform('count')
        #print(self.dfx.head)
        mask = (self.dfx['date'] >= datef) & (self.dfx['date'] <= datet) & (self.dfx["num_ratings"]>= self.min_ratings)
        #####print(self.dfx.loc[mask].sort_values(by=['date'],ascending=False))
    fin = self.dfx.loc[mask]
    return fin
    



import pandas as pd
from datetime import datetime as dtm

class MovieData:
     def __init__(self, path):
        self.path = path
        self.df = pd.read_table(path, encoding = 'ISO-8859-1')
     def get_title(self, id):
        x = self.df["title"][self.df.id == id]
        return self.df[self.df["id"] == id].title.item()
        #return self.df[self.df["id"] == id]["title"]
         



from random import randrange
import numpy as np
class RandomPredictor:
    def __init__(self, minimum, maximum):
        self.maximum = maximum
        self.minimum = minimum
    def fit(self, x:UserItemData):
        self.x=x
        #print(type(self.x))
    def predict(self, user_id):
        self.user_id = user_id
        mydict = {}
        numratings = self.x.nratings()
                         #print(type(x))
                                 
                                         #for i in range(1, numratings):
        for i in self.x.df["movieID"].unique():
             mydict[i] = randrange(self.minimum, self.maximum+1)
        return mydict
        


import math
class Recommender:
    def __init__(self, pr):
        self.pr = pr
    def fit(self, x):
        self.x = x
        self.pr.fit(self.x)
    def recommend(self, userID, n = 10, rec_seen = True):
         self.pr.fit(self.x)
         y = self.pr.predict(userID)
         dic = dict(sorted(y.items(), key=lambda item: item[1], reverse=True))
         num = n
         recomm = {}
         if rec_seen==True:
             val = list(dic.values())
             #k = len(val)
             key = list(dic.keys())
             for x in range(num+1):
                 recomm[key[x]] = val[x]
         else:
             val = list(dic.values())
             #k = len(val)
             key = list(dic.keys())
             i=0
             k=0
             while (k < n) & (i<len(key)):
                 if ((self.x.df['movieID'] == key[i]) & (self.x.df['userID'] == userID)).any() == True:
                     i=i+1
                 else:
                     recomm[key[i]] = val[i]
                     i=i+1
                     k=k+1
         return recomm
    def evaluate(self, test_data, n):
        #MAE and RMSE
        upp_mae = 0
        upp_rmse = 0
        count = 0
        
        n_cnt = 0
        avarage = dict(test_data.groupby('movieID')['rating'].mean())
        for user in test_data["userID"].unique():
            userpred = self.pr.predict(user)
            for key, value in userpred.items():
                if ((test_data['movieID'] == key) & (test_data['userID'] == user)).any() == True:
                    upp_mae = upp_mae + abs(value - test_data[(test_data["userID"]==user) & (test_data["movieID"]==key)].rating.item())
                    upp_rmse = upp_rmse + ((value - test_data[(test_data["userID"]==user) & (test_data["movieID"]==key)].rating.item()) ** 2)
                    count = count + 1
        mae = upp_mae / count
        rmse = math.sqrt(upp_rmse / count)

        #precision
        falsepos = 0
        falseneg = 0
        truepos = 0
        trueneg = 0
        for user in test_data["userID"].unique():
            add_ratings = {}
            recc_ratings = {}
            recomenders = test_data.predict(user)
            for key, value in recomenders.items():
                if (value > avarage[key]):
                    recc_ratings[key] = value

            for index, row in test_data.iterrows():
                if (row["userID"]==user) & (row["rating"] > avarage[row["movieID"]]):
                    add_ratings[row["movieID"]] = row["rating"]
            
            add_ratings = dict(sorted(add_ratings.items(), key=lambda item: item[1], reverse=True))
            add_ratings = {k: add_ratings[k] for k in list(add_ratings.keys())[:n]}

            recc_ratings = dict(sorted(recc_ratings.items(), key=lambda item: item[1], reverse=True))
            recc_ratings = {k: recc_ratings[k] for k in list(recc_ratings.keys())[:n]}

            shared_items = {k: add_ratings[k] for k in add_ratings if k in recc_ratings}
            not_shared_items1 = {k: add_ratings[k] for k in add_ratings if k not in recc_ratings}
            not_shared_items2 = {k: recc_ratings[k] for k in recc_ratings if k not in add_ratings}
            falsepos = falsepos + len(not_shared_items2)
            falseneg = falsepos + len(not_shared_items1)
            truepos = truepos + len(shared_items)
        precision = truepos/(truepos+falsepos)
        recall = truepos/(truepos+falseneg)
        accuracy = truepos / n
        f1 = 2*((precision*recall)/(precision+recall))
            
        return (mae, rmse, precision, recall, accuracy, f1)





                 
class ViewsPredictor:
    def __init__(self, n=5):
        self.n = n
    def fit(self, x):
        self.x = x
    def predict(self, user_id, rec_seen):
        self.user_id = user_id
        self.rec_seen = rec_seen
        #for i in self.x.df["movieID"].unique():
        d = dict(self.x.df.movieID.value_counts())
        keys = list(d.keys())
        values = list(d.values())
        views = {}
        #MAX_VALUES=self.n
        #d = {k: d[k] for k in d.keys()[:MAX_VALUES]}
        if self.rec_seen == True:
            d = {k: d[k] for k in list(d.keys())[:self.n]}
            return d
        else: 
            i=1
            count = 0
            while count < self.n:
                if ((self.x.df['movieID'] == keys[i]) & (self.x.df['userID'] == user_id)).any() == True:
                    i=i+1
                else:
                    views[keys[i]] = values[i]
                    count = count +1
                    i=i+1
            return views
         


class AvaragePredictor:
    def __init__(self, b):
        self.b = b
    def fit(self, x, n, user_id, rec_seen):
        self.rec_seen = rec_seen
        self.n = n
        self.x = x
        self.user_id = user_id
        summed = dict(self.x.df.groupby('movieID')['rating'].sum())
        avarage = self.x.df['rating'].mean()
        numscores = {}
        pred = {}
        ret = {}
        for i in self.x.df["movieID"].unique():
            numscores[i] = (self.x.df.movieID == i).sum()
            pred[i] = (summed[i] + self.b * avarage)/(numscores[i] + self.b)
            #x = self.df["title"][self.df.id == id]
            #print(self.x.df["movieID"==i]["rating"])
            #print(self.x.df[self.x.df["movieID"] == i].rating.item())
        pred = dict(sorted(pred.items(), key=lambda item: item[1], reverse=True))
        if self.rec_seen == True:
            return {k: pred[k] for k in list(pred.keys())[:self.n]}
        else: 
            i=1
            for i in pred.keys():
                if ((self.x.df['movieID'] == i) & (self.x.df['userID'] == user_id)).any() == True:
                    i=i+1
                else:
                    ret[i] = pred[i]
            #print(ret[self.x.df["movieID"][self.df.title == "Sonnenallee"]])
            return {k: ret[k] for k in list(ret.keys())[:self.n]}      


import math
class ItemBasedPredictor:
    def __init__(self, min_values=0, threshold=0):
        self.min_values = min_values
        self.threshold = threshold
    def fit(self, x):
        self.x = x
        self.avarage = dict(self.x.df.groupby('userID')['rating'].mean())
        #print(self.avarage)
    def predict(self, user_id):
        preds = {}
        rated = {}
        self.users = self.x.df["userID"].unique()
        self.movies = self.x.df["movieID"].unique()
        #rt = dict(self.x.df[(self.x.df["userID"]==user_id)].rating)
        userrated = self.x.df[(self.x.df["userID"]==user_id)]
        rt = dict(zip(userrated.movieID, userrated.rating))
        for movie in self.x.df["movieID"].unique():
            up = 0
            down = 0
            for mv, rating in rt.items():
                up = up + (self.similarity(movie, mv) * rating)
                down = down + self.similarity(movie, mv)
            if down > 0:
                preds[movie] = up/down
            else:
                preds[movie] = self.avarage[user_id]
        
        return preds
    def similarItems(self, item, n):
        similar = {}
        for movie in self.x.df["movieID"].unique():
            if item != movie:
                similar[movie] = self.similarity(item, movie)
        dic = dict(sorted(similar.items(), key=lambda item: item[1], reverse=True))
        return ({k: dic[k] for k in list(dic.keys())[:n]})

    def similarity(self,p1,p2):
        cnt = 0
        first = 0
        second = 0
        users = []
       
        for i in self.x.df["userID"].unique():
            if (((self.x.df['userID'] == i) & (self.x.df['movieID'] == p1)).any() == True) & (((self.x.df['userID'] == i) & (self.x.df['movieID'] == p2)).any() == True):
                users.append(i)
                av = self.avarage[i]
                #return self.df[self.df["id"] == id].title.item()
                m1 = self.x.df[(self.x.df["userID"]==i) & (self.x.df["movieID"]==p1)].rating.item()
                m2 = self.x.df[(self.x.df["userID"]==i) & (self.x.df["movieID"]==p2)].rating.item()
                cnt = cnt + (( m1- av)*(m2 - av))
                first = first + ((m1 - av)*(m1 - av))
                second = second + ((m2 - av)*(m2 - av))
        if (math.sqrt(first)* math.sqrt(second)) > 0:
            res = cnt / (math.sqrt(first)* math.sqrt(second))
        else:
            res = 0
        if len(users)<self.min_values:
            return 0   
        if res < self.threshold:
            return 0
        return res
    



def reccMyself():
    my = UserItemData("data/mydata.dat")
    uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1300)
    rp = ItemBasedPredictor()
    rec = Recommender(rp)
    rec.fit(uim)
    final = pd.concat([my.df, uim.df], ignore_index=True, sort=False)
    recommended = rec.recommend(1111111, n=5, rec_seen=False)
    return recommended
    


def most_similar(x):
    dic = {}
    movies = x.df["movieID"].unique()
    for moviei in movies:
        for moviej in movies:
            tp1 = (moviei, moviej)
            tp2 = (moviej, moviei)
            if (moviei != moviej) & (tp1 not in dic) & (tp2 not in dic):
                dic[tp1] = rp.similarity(moviei, moviej)
    x = {k: dic[k] for k in list(dic.keys())[:10]}
    for movi,movj in x:
        print("Movies: {} and {}, score: {}".format(md.get_title(movi),md.get_title(movj), x[(movi,movj)]))





class SlopeOnePredictor:
    def __init__(self, min_values=0, threshold=0):
        self.min_values = min_values
        self.threshold = threshold
    def fit(self, x):
        self.x = x
    def predict(self, user_id):
        pred = {}
        newx = self.x.df.to_dict()
        self.users = self.x.df["userID"].unique()
        self.movies = self.x.df["movieID"].unique()
        rt = self.x.df[(self.x.df["userID"]==user_id)]
        mydict = dict(zip(rt.movieID, rt.rating))
        for moviej in self.movies:
            up = 0
            down = 0
            for moviei, rating in mydict.items():
                deviation = self.dev(moviej, moviei)
                up = up + ((deviation[0] + rating)*deviation[1])
                down = down + deviation[1]
            pred[moviej] = up/down
        return pred

    def dev(self, j, i):
        up = 0
        count = 0
        for user in self.users:
            if (((self.x.df['userID'] == user) & (self.x.df['movieID'] == j)).any() == True) & (((self.x.df['userID'] == user) & (self.x.df['movieID'] == i)).any() == True):
                ratingj = self.x.df[(self.x.df["userID"]==user) & (self.x.df["movieID"]==j)].rating.item()
                ratingi = self.x.df[(self.x.df["userID"]==user) & (self.x.df["movieID"]==i)].rating.item()
                up = up + (ratingj - ratingi)
                count = count + 1
        z = [(up / count), count]
        return z





##results
##useritemdata
print("UserItemData:")
x = UserItemData("data/user_ratedmovies.dat", from_date = '12.1.2007', to_date='16.2.2008',min_ratings=1000)
print(x.nratings())
y = UserItemData("data/user_ratedmovies.dat")
print(y.nratings())
#moviedata
print("MovieData:")
md = MovieData('data/movies.dat')
print(md.get_title(1))
#randompredictor
print("RandomPredictor:")
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat') 
rp = RandomPredictor(1, 5) 
rp.fit(uim) 
pred = rp.predict(78) 
print(type(pred)) 
items = [1, 3, 20, 50, 100] 
for item in items: 
    print("Movie: {}, score: {}".format(md.get_title(item), pred[item]))
#recommender
print("Recommender:")
md = MovieData('data/movies.dat') 
uim = UserItemData('data/user_ratedmovies.dat') 
rp = RandomPredictor(1, 5) 
rec = Recommender(rp) 
rec.fit(uim) 
rec_items = rec.recommend(78, n=7, rec_seen=False) 
for idmovie, val in rec_items.items(): 
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))
#avaragepredictor
print("AveragePredictor:")
uim = UserItemData('data/user_ratedmovies.dat')
av = AvaragePredictor(0)
av_pr = av.fit(uim, 10, 78, False)
for idmovie, val in av_pr.items(): 
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))
#ViewsPredictor
print("ViewsPredictor")
md = MovieData('data/movies.dat') 
uim = UserItemData('data/user_ratedmovies.dat')
vp = ViewsPredictor()
vp.fit(uim)
most_watched = vp.predict(78, False)
for idmovie, val in most_watched.items(): 
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))
#similarities
print("similars: ")
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)
print("Similarity between the movies 'Men in black'(1580) and 'Ghostbusters'(2716): ", rp.similarity(1580, 2716))
print("Similarity between the movies 'Men in black'(1580) and 'Schindler's List'(527): ", rp.similarity(1580, 527))
print("Similarity between the movies 'Men in black'(1580) and 'Independence day'(780): ", rp.similarity(1580, 780))
#SlopeOnePredictor
print("slope one")
md = MovieData('data/movies.dat') 
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1300) 
rp = SlopeOnePredictor() 
rec = Recommender(rp) 
rec.fit(uim)
print("Predictions for 78: ") 
rec_items = rec.recommend(78, n=5, rec_seen=False) 
for idmovie, val in rec_items.items():
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))




#useritembased
print("userItemData")
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1300)
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)
print("Predictions for 78: ")
rec_items = rec.recommend(78, n=15, rec_seen=False)
for idmovie, val in rec_items.items():
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))

#similaritems
rec_items = rp.similarItems(4993, 5)
print('Movies similar to "The Lord of the Rings: The Fellowship of the Ring": ')
for idmovie, val in rec_items.items():
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))

UserItemData:
22021
855598
MovieData:
Toy story
RandomPredictor:
<class 'dict'>
Movie: Toy story, score: 4
Movie: Grumpy Old Men, score: 4
Movie: Money Train, score: 4
Movie: The Usual Suspects, score: 1
Movie: City Hall, score: 5
Recommender:
Movie: Grumpy Old Men, score: 5
Movie: The Crow, score: 5
Movie: Last Man Standing, score: 5
Movie: Payback, score: 5
Movie: Superman, score: 5
Movie: The General's Daughter, score: 5
Movie: South Park: Bigger Longer & Uncut, score: 5
AveragePredictor:
Movie: Brother Minister: The Assassination of Malcolm X, score: 5.0
Movie: Synthetic Pleasures, score: 5.0
Movie: Adam & Steve, score: 5.0
Movie: Gabbeh, score: 5.0
Movie: Eve and the Fire Horse, score: 5.0
Movie: Saikaku ichidai onna, score: 5.0
Movie: Maradona by Kusturica, score: 5.0
Movie: The Princess and the Cobbler, score: 5.0
Movie: Vals Im Bashir, score: 5.0
Movie: Ko to tamo peva, score: 5.0
ViewsPredictor
Movie: The Lord of the Rings: The Fellowship of the Ring, score: 1576
Movie: The Lo

KeyboardInterrupt: 